In [19]:
import os

# takes care of annoying TF-GPU warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# remove useless Tensorflow warning:
# WARNING:absl:Found untraced functions such as _update_step_xla, lstm_cell_1_layer_call_fn, 
# lstm_cell_1_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_fn, 
# lstm_cell_2_layer_call_and_return_conditional_losses while saving (showing 5 of 5). 
# These functions will not be directly callable after loading.
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [20]:
# very useful for managing wandb runs: https://stackoverflow.com/questions/71106179/log-two-model-runs-with-keras-wandb
import wandb
from wandb.keras import WandbCallback
os.environ["WANDB_SILENT"] = "true"

In [21]:
import multi_classifier_utils as mc_u

#### Logistic Regression: Formula Label Prediction (multi-label, all features)

In [22]:
import numpy as np 
import pandas as pd
from pathlib import Path 
import ast

import tensorflow as tf
import keras_tuner as kt

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

import datetime

tfds.disable_progress_bar()
wandb_project_name = "multi_label_formula_classification"

In [23]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_"+metric], "")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, "val_"+metric])

#### Import Data and Preprocess Dataframe

In [24]:
def preprocess_data(corpus,
                    irrelevant_features=["mtype",]):
    # drop irrelevant columns
    corpus.drop(irrelevant_features, inplace=True, axis=1)

    def cell_str_to_list(cell_val):
        return ast.literal_eval(cell_val)

    # filter strings
    def process_cell(cell_str):
        stripped_f_str = cell_str[1:-1].replace("\\\\", "\\")
        f_list = stripped_f_str.split(",")
        f_list = [token.replace("'", "").replace(" ", "") for token in f_list]
        f_list = ["{" if token == "\\{" else token for token in f_list]
        f_list = ["}" if token == "\\}" else token for token in f_list]
        cell_str = " ".join(f_list)
        return cell_str

    corpus["type_tokens"] = corpus["type_tokens"].map(process_cell)
    corpus["tokens"] = corpus["tokens"].map(process_cell)
    corpus["mtype_one_hot"] = corpus["mtype_one_hot"].map(cell_str_to_list)
    corpus["labels"] = corpus["labels"].map(cell_str_to_list)
    corpus = corpus.loc[(corpus["tokens"].str.len() > 0) & (corpus["tokens"] != " ")]


In [25]:
#print(os.getcwd())
data_p = Path("../data/") / "multi_class_unbalanced_data_TOKENIZED_V1.csv"
data = pd.read_csv(data_p)
preprocess_data(data)
data.head()

,tokens,type_tokens,labels,labels_str,mtype_one_hot
0,f,func_name __ANON_1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
1,j : \mathbb{N} \rightarrow [ 0 1 ],func_def func_name __ANON_1 COLON mapping set_...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
2,b,func_name __ANON_1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
3,mathbb{Q,set_constant SET_BASIC,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[1, 0, 0]"
4,f,func_name __ANON_1,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","['functions', 'elementary-set-theory']","[0, 0, 1]"


In [26]:
print(data["type_tokens"].map(lambda x: len((x.split(" ")))).max())
print(data["tokens"].map(lambda x: len((x.split(" ")))).max())

254
252


In [27]:
# ordinary datasets
SMALL_TRAIN_SIZE = 24620 - 2460
SMALL_TEST_SIZE = 2460
LARGE_TRAIN_SIZE = 106523 - 10650
LARGE_TEST_SIZE = 10650
# compact datasets
NUM_CLASSES = 40

labels_array = np.array(data["labels"].to_list())
m_type_array = np.array(data["mtype_one_hot"].to_list())

dataset4_all_features = tf.data.Dataset.from_tensor_slices((data["tokens"],data["type_tokens"],m_type_array), name="data")
dataset3_tokens_types = tf.data.Dataset.from_tensor_slices((data["tokens"],data["type_tokens"]), name="data")
dataset2_types = tf.data.Dataset.from_tensor_slices((data["type_tokens"]), name="data")
dataset1_tokens = tf.data.Dataset.from_tensor_slices((data["tokens"]), name="data")

labels_ds = tf.data.Dataset.from_tensor_slices(labels_array, name="label")
#data_as_ds = tf.data.Dataset.zip((dat_as_ds, labels_ds))

dataset1_tokens_l = tf.data.Dataset.zip((dataset1_tokens, labels_ds))
dataset2_types_l = tf.data.Dataset.zip((dataset2_types, labels_ds))
dataset3_tokens_types_l = tf.data.Dataset.zip((dataset3_tokens_types, labels_ds))
dataset4_all_features_l = tf.data.Dataset.zip((dataset4_all_features, labels_ds))

In [28]:
test_dataset1 = dataset1_tokens_l.take(SMALL_TEST_SIZE)
train_dataset1 = dataset1_tokens_l.skip(SMALL_TEST_SIZE)
test_dataset2 = dataset2_types_l.take(SMALL_TEST_SIZE)
train_dataset2 = dataset2_types_l.skip(SMALL_TEST_SIZE)
test_dataset3 = dataset3_tokens_types_l.take(SMALL_TEST_SIZE)
train_dataset3 = dataset3_tokens_types_l.skip(SMALL_TEST_SIZE)
test_dataset4 = dataset4_all_features_l.take(SMALL_TEST_SIZE)
train_dataset4 = dataset4_all_features_l.skip(SMALL_TEST_SIZE)

##### Setup and Data Preparation

In [29]:
for (example_token, example_type, example_m_type), label in train_dataset4.take(5):
    print("text: ", example_token.numpy())
    print("type: ", example_type.numpy())
    print("m_type: ", example_m_type.numpy())
    print("label: ", label.numpy())

text:  b'{ 2   4 }'
type:  b'explset L_BRACE_LITERAL set_enumeration item __ANON_3 COMMA item __ANON_3 R_BRACE_LITERAL'
m_type:  [1 0 0]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f : I \\to X'
type:  b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f ( a ) = 1'
type:  b'func_expr func_name_arg func_name __ANON_1 L_PAREN __ANON_1 R_PAREN EQUAL expr_atom __ANON_3'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'x'
type:  b'func_name __ANON_1'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f : X \\to Y'
type:  b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [30]:
BUFFER_SIZE = 2000
BATCH_SIZE = 64
STEPS_PER_EPOCH = np.floor(SMALL_TRAIN_SIZE/BATCH_SIZE)
VAL_STEPS_PER_EPOCH = np.floor(SMALL_TEST_SIZE/BATCH_SIZE)
#train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
#test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [31]:
test_dataset1 = test_dataset1.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset1 = train_dataset1.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset2 = test_dataset2.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset2 = train_dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset3 = test_dataset3.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset3 = train_dataset3.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset4 = test_dataset4.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset4 = train_dataset4.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

##### Text Encoding

In [32]:
# for int encoder
TYPE_TOKENS_MAX_SEQ_LEN = 260
TOKENS_MAX_SEQ_LEN = 260


# for other encoders 
TYPE_TOKENS_PAD_TO_MAX_TOKENS = 80
TOKENS_PAD_TO_MAX_TOKENS = 200
BIGRAM_PAD_TO_MAX_TOKENS = 350

In [33]:
def adapt_encoder(encoder, mode, dataset, dataset_type):
    if mode == "token": 

        if dataset_type == 1:
            encoder.adapt(dataset.map(lambda inputs, label: inputs))
        elif dataset_type == 2:
            ...
        elif dataset_type == 3:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text
        elif dataset_type == 4:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text  
    elif mode == "type":
        
        if dataset_type == 1:
            ...
        elif dataset_type == 2:
            encoder.adapt(dataset.map(lambda inputs, label: inputs))
        elif dataset_type == 3:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text
        elif dataset_type == 4:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text

    return encoder

In [34]:
def create_encoder(output_mode_str, n_grams, mode, dataset, dataset_type):
    if output_mode_str == "int":
        VOCAB_SIZE = 200
        if mode == "token":
            max_seq_len = TOKENS_MAX_SEQ_LEN
        elif mode == "type":
            max_seq_len = TYPE_TOKENS_MAX_SEQ_LEN

        encoder = tf.keras.layers.TextVectorization(
            standardize=None,
            output_mode=output_mode_str,
            ngrams = n_grams,
            output_sequence_length = max_seq_len,
            split="whitespace",
            max_tokens=VOCAB_SIZE)
        #TODO: adapt for different inputs
        encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
        return encoder
    
    if output_mode_str == "count" and n_grams == 2:
        max_seq_len = BIGRAM_PAD_TO_MAX_TOKENS
        encoder = tf.keras.layers.TextVectorization(
            standardize=None,
            output_mode=output_mode_str,
            ngrams = n_grams,
            pad_to_max_tokens = max_seq_len,
            split="whitespace",
            max_tokens=max_seq_len)
        
        
        encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
        return encoder
    
    if mode == "token":
        max_seq_len = TOKENS_PAD_TO_MAX_TOKENS
    elif mode == "type":
        max_seq_len = TYPE_TOKENS_PAD_TO_MAX_TOKENS

    encoder = tf.keras.layers.TextVectorization(
        standardize=None,
        output_mode=output_mode_str,
        ngrams = n_grams,
        pad_to_max_tokens = max_seq_len,
        split="whitespace",
        max_tokens=max_seq_len)
    #TODO: adapt for different inputs
    encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
    return encoder


    

##### Representation 1: Use integer indices encoding

In [35]:
encoder_int_tokens = create_encoder("int", None, "token", train_dataset1, 1)
encoder_int_types = create_encoder("int", None, "type", train_dataset2, 2)

vocab_tokens = np.array(encoder_int_tokens.get_vocabulary())
vocab_size_tokens = len(encoder_int_tokens.get_vocabulary())
vocab_types = np.array(encoder_int_types.get_vocabulary())
vocab_size_types = len(encoder_int_types.get_vocabulary())

print("tokens (voc size): ", vocab_size_tokens)
print("types (voc size): ", vocab_size_types)

tokens (voc size):  128
types (voc size):  61


In [36]:
encoded_example_token = encoder_int_tokens(example_token).numpy()
encoded_example_types = encoder_int_types(example_type).numpy()

print("tokens: ")
print(example_token)
print(encoded_example_token)
print(encoded_example_token.shape)

print("types: ")
print(example_type)
print(encoded_example_types)
print(encoded_example_types.shape)

tokens: 
tf.Tensor(b'f : X \\to Y', shape=(), dtype=string)
[ 2 11 26 14 35  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
(260,)
types: 
tf.Tensor(b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0', shape=(), dtype=string)
[15  5  3 13 14  6 16  6  0  0  

##### Representation 2: Count Vectorizer

In [37]:
encoder_count_tokens = create_encoder("count", None, "token", train_dataset1, 1)
encoder_count_types = create_encoder("count", None, "type", train_dataset2, 2)

vocab_tokens = np.array(encoder_count_tokens.get_vocabulary())
vocab_size_tokens = len(encoder_count_tokens.get_vocabulary())
vocab_types = np.array(encoder_count_types.get_vocabulary())
vocab_size_types = len(encoder_count_types.get_vocabulary())

print("tokens (voc size): ", vocab_size_tokens)
print("types (voc size): ", vocab_size_types)

tokens (voc size):  127
types (voc size):  60


In [38]:
encoded_example_token = encoder_count_tokens(example_token).numpy()
encoded_example_types = encoder_count_types(example_type).numpy()

print("tokens: ")
print(example_token)
print(encoded_example_token)
print(encoded_example_token.shape)

print("types: ")
print(example_type)
print(encoded_example_types)
print(encoded_example_types.shape)

tokens: 
tf.Tensor(b'f : X \\to Y', shape=(), dtype=string)
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
(200,)
types: 
tf.Tensor(b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0', shape=(), dtype=string)
[0. 0. 1. 0. 1. 2. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

##### Model (Logistic Regression)

 **Train the model**

#### Experiment 1: Use integer indices for encoding tokens

##### Model 1: ONLY TOKENS
Find best hyperparameters

In [39]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens1 = create_encoder("int", None, "token", train_dataset1, 1)
model_builder1 = mc_u.create_model_builder1_LR(NUM_CLASSES, encoder_int_tokens1, tokens_input_len)

In [40]:
tuner = kt.Hyperband(model_builder1,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model1_lr',
                     project_name='model1_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [41]:
tuner.search(train_dataset1,
             epochs=30,
             validation_data=test_dataset1,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 00m 44s]
val_accuracy: 0.2331414520740509

Best val_accuracy So Far: 0.8725329041481018
Total elapsed time: 00h 12m 53s
INFO:tensorflow:Oracle triggered exit


In [42]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
96
0.2
0.3


Train with best hyperparameters

In [43]:
NUM_EPOCHS = 80
model1 = mc_u.create_model1_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_int_tokens1, tokens_input_len)
model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [44]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_only_tokens",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model1.fit(train_dataset1, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset1,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
344/346 [============================>.] - ETA: 0s - loss: 0.6459 - accuracy: 0.0849 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


346/346 [==============================] - 5s 15ms/step - loss: 0.6456 - accuracy: 0.0861 - recall_1: 0.0000e+00 - val_loss: 0.6061 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/80
341/346 [============================>.] - ETA: 0s - loss: 0.5073 - accuracy: 0.2422 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


346/346 [==============================] - 5s 14ms/step - loss: 0.5061 - accuracy: 0.2436 - recall_1: 0.0000e+00 - val_loss: 0.4540 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/80
344/346 [============================>.] - ETA: 0s - loss: 0.3460 - accuracy: 0.2890 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


...

346/346 [==============================] - 5s 13ms/step - loss: 0.1337 - accuracy: 0.5886 - recall_1: 0.1308 - val_loss: 0.1210 - val_accuracy: 0.7738 - val_recall_1: 0.2196
Epoch 79/80
343/346 [============================>.] - ETA: 0s - loss: 0.1332 - accuracy: 0.5915 - recall_1: 0.1328INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


346/346 [==============================] - 5s 15ms/step - loss: 0.1335 - accuracy: 0.5890 - recall_1: 0.1322 - val_loss: 0.1210 - val_accuracy: 0.7738 - val_recall_1: 0.2196
Epoch 80/80
346/346 [==============================] - ETA: 0s - loss: 0.1335 - accuracy: 0.5890 - recall_1: 0.1314INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


346/346 [==============================] - 5s 14ms/step - loss: 0.1335 - accuracy: 0.5890 - recall_1: 0.1314 - val_loss: 0.1209 - val_accuracy: 0.7743 - val_recall_1: 0.2196


##### Model 2: ONLY TYPES
Find best hyperparameters

In [50]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_types2 = create_encoder("int", None, "type", train_dataset2, 2)    
model_builder2 = mc_u.create_model_builder2_LR(NUM_CLASSES, encoder_int_types2,type_input_len)

In [51]:
tuner = kt.Hyperband(model_builder2,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model2_lr',
                     project_name='model2_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


INFO:tensorflow:Reloading Tuner from meta_dir/model2_lr/model2_lr/tuner0.json


INFO:tensorflow:Reloading Tuner from meta_dir/model2_lr/model2_lr/tuner0.json


In [52]:
tuner.search(train_dataset2,
             epochs=30,
             validation_data=test_dataset2,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 25 Complete [00h 01m 15s]
val_accuracy: 0.5353618264198303

Best val_accuracy So Far: 0.875
Total elapsed time: 00h 11m 07s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [53]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
64
0.2
0.3


Train with best hyperparameters

In [54]:
NUM_EPOCHS = 80
model2 = mc_u.create_model2_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_int_types2, type_input_len)
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [55]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_only_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model2.fit(train_dataset2, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset2,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
341/346 [============================>.] - ETA: 0s - loss: 0.6561 - accuracy: 0.0669 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_235239-39ogj0p4/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_235239-39ogj0p4/files/model-best/assets


346/346 [==============================] - 4s 10ms/step - loss: 0.6554 - accuracy: 0.0662 - recall_1: 0.0000e+00 - val_loss: 0.6184 - val_accuracy: 0.2410 - val_recall_1: 0.0000e+00
Epoch 2/80
345/346 [============================>.] - ETA: 0s - loss: 0.5360 - accuracy: 0.2428 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_235239-39ogj0p4/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_235239-39ogj0p4/files/model-best/assets


346/346 [==============================] - 3s 10ms/step - loss: 0.5358 - accuracy: 0.2422 - recall_1: 0.0000e+00 - val_loss: 0.4843 - val_accuracy: 1.0000 - val_recall_1: 0.0000e+00
Epoch 3/80
345/346 [============================>.] - ETA: 0s - loss: 0.3834 - accuracy: 0.3574 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_235239-39ogj0p4/files/model-best/assets


...

346/346 [==============================] - 3s 10ms/step - loss: 0.1346 - accuracy: 0.5896 - recall_1: 0.1163 - val_loss: 0.1214 - val_accuracy: 0.8713 - val_recall_1: 0.2717
Epoch 74/80
345/346 [============================>.] - ETA: 0s - loss: 0.1343 - accuracy: 0.5899 - recall_1: 0.1151INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_235239-39ogj0p4/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_235239-39ogj0p4/files/model-best/assets


346/346 [==============================] - 4s 10ms/step - loss: 0.1343 - accuracy: 0.5891 - recall_1: 0.1150 - val_loss: 0.1212 - val_accuracy: 0.8692 - val_recall_1: 0.2593
Epoch 75/80
346/346 [==============================] - 2s 7ms/step - loss: 0.1344 - accuracy: 0.5884 - recall_1: 0.1157 - val_loss: 0.1213 - val_accuracy: 0.8692 - val_recall_1: 0.2593
Epoch 76/80
346/346 [==============================] - 3s 7ms/step - loss: 0.1343 - accuracy: 0.5873 - recall_1: 0.1165 - val_loss: 0.1213 - val_accuracy: 0.8697 - val_recall_1: 0.2593
Epoch 77/80
346/346 [==============================] - 3s 8ms/step - loss: 0.1345 - accuracy: 0.5870 - recall_1: 0.1158 - val_loss: 0.1213 - val_accuracy: 0.8697 - val_recall_1: 0.2593
Epoch 78/80
346/346 [==============================] - 3s 8ms/step - loss: 0.1345 - accuracy: 0.5899 - recall_1: 0.1156 - val_loss: 0.1213 - val_accuracy: 0.8697 - val_recall_1: 0.2593
Epoch 79/80
346/346 [==============================] - 3s 7ms/step - loss: 0.1343 - ac

INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_235239-39ogj0p4/files/model-best/assets


346/346 [==============================] - 3s 10ms/step - loss: 0.1342 - accuracy: 0.5914 - recall_1: 0.1159 - val_loss: 0.1211 - val_accuracy: 0.8697 - val_recall_1: 0.2593


##### Model 3: TOKENS AND TYPES
Find best hyperparameters

In [56]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens3 = create_encoder("int", None, "token", train_dataset3, 3)
encoder_int_types3 = create_encoder("int", None, "type", train_dataset3, 3)
model_builder3 = mc_u.create_model_builder3_LR(NUM_CLASSES, encoder_int_tokens3,encoder_int_types3, tokens_input_len, type_input_len)

In [57]:
tuner = kt.Hyperband(model_builder3,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model3_lr',
                     project_name='model3_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [58]:
tuner.search(train_dataset3,
             epochs=30,
             validation_data=test_dataset3,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 01m 32s]
val_accuracy: 0.8651315569877625

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 21m 39s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [59]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
32
0.3
0.2


Train with best hyperparameters

In [60]:
NUM_EPOCHS = 80
model3 = mc_u.create_model3_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_int_tokens3, encoder_int_types3, tokens_input_len, type_input_len)
model3.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [61]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_tokens_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model3.fit(train_dataset3, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset3,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
343/346 [============================>.] - ETA: 0s - loss: 0.6650 - accuracy: 0.2317 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


346/346 [==============================] - 4s 11ms/step - loss: 0.6648 - accuracy: 0.2322 - recall_1: 0.0000e+00 - val_loss: 0.6429 - val_accuracy: 0.0037 - val_recall_1: 0.0000e+00
Epoch 2/80
346/346 [==============================] - ETA: 0s - loss: 0.5854 - accuracy: 0.2940 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


346/346 [==============================] - 4s 10ms/step - loss: 0.5854 - accuracy: 0.2940 - recall_1: 0.0000e+00 - val_loss: 0.5605 - val_accuracy: 0.2183 - val_recall_1: 0.0000e+00
Epoch 3/80
341/346 [============================>.] - ETA: 0s - loss: 0.4749 - accuracy: 0.3247 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


...

346/346 [==============================] - 4s 12ms/step - loss: 0.1380 - accuracy: 0.5743 - recall_1: 0.0889 - val_loss: 0.1297 - val_accuracy: 0.8487 - val_recall_1: 0.2016
Epoch 79/80
344/346 [============================>.] - ETA: 0s - loss: 0.1376 - accuracy: 0.5743 - recall_1: 0.0903INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


346/346 [==============================] - 4s 10ms/step - loss: 0.1378 - accuracy: 0.5732 - recall_1: 0.0902 - val_loss: 0.1297 - val_accuracy: 0.8487 - val_recall_1: 0.2016
Epoch 80/80
339/346 [============================>.] - ETA: 0s - loss: 0.1374 - accuracy: 0.5802 - recall_1: 0.0922INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_001902-rz5gyg0j/files/model-best/assets


346/346 [==============================] - 4s 10ms/step - loss: 0.1379 - accuracy: 0.5755 - recall_1: 0.0916 - val_loss: 0.1296 - val_accuracy: 0.8487 - val_recall_1: 0.2016


##### Model 4: TOKENS AND TYPES AND SEM MATH LABELS
Find best hyperparameters

In [62]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens4 = create_encoder("int", None, "token", train_dataset4, 4)
encoder_int_types4 = create_encoder("int", None, "type", train_dataset4, 4)
model_builder4 = mc_u.create_model_builder4_LR(NUM_CLASSES, encoder_int_tokens4,encoder_int_types4, tokens_input_len, type_input_len, "int")

In [63]:
tuner = kt.Hyperband(model_builder4,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model4_lr',
                     project_name='model4_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [64]:
tuner.search(train_dataset4,
             epochs=30,
             validation_data=test_dataset4,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 00m 49s]
val_accuracy: 0.12212171405553818

Best val_accuracy So Far: 0.9379112124443054
Total elapsed time: 00h 18m 44s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [65]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
128
0.1
0.3


Train with best hyperparameters

In [66]:
NUM_EPOCHS = 80
model4 = mc_u.create_model4_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_int_tokens4, encoder_int_types4, tokens_input_len, type_input_len, "int")
model4.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [67]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_all_features",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model4.fit(train_dataset4, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset4,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
346/346 [==============================] - ETA: 0s - loss: 0.6187 - accuracy: 0.1947 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


346/346 [==============================] - 12s 32ms/step - loss: 0.6187 - accuracy: 0.1947 - recall_1: 0.0000e+00 - val_loss: 0.5387 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/80
345/346 [============================>.] - ETA: 0s - loss: 0.4158 - accuracy: 0.3204 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


346/346 [==============================] - 11s 32ms/step - loss: 0.4155 - accuracy: 0.3201 - recall_1: 0.0000e+00 - val_loss: 0.3327 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/80
346/346 [==============================] - ETA: 0s - loss: 0.2526 - accuracy: 0.3217 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


...

346/346 [==============================] - 12s 34ms/step - loss: 0.1336 - accuracy: 0.5979 - recall_1: 0.1195 - val_loss: 0.1197 - val_accuracy: 0.8331 - val_recall_1: 0.2203
Epoch 79/80
346/346 [==============================] - ETA: 0s - loss: 0.1336 - accuracy: 0.5977 - recall_1: 0.1201INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


346/346 [==============================] - 12s 34ms/step - loss: 0.1336 - accuracy: 0.5977 - recall_1: 0.1201 - val_loss: 0.1197 - val_accuracy: 0.8331 - val_recall_1: 0.2203
Epoch 80/80
346/346 [==============================] - ETA: 0s - loss: 0.1336 - accuracy: 0.5976 - recall_1: 0.1197INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_004351-4q2gum65/files/model-best/assets


346/346 [==============================] - 12s 34ms/step - loss: 0.1336 - accuracy: 0.5976 - recall_1: 0.1197 - val_loss: 0.1196 - val_accuracy: 0.8339 - val_recall_1: 0.2201


#### Experiment 2: Use count vectorizer

##### Model 1: ONLY TOKENS
Find best hyperparameters

In [68]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens1 = create_encoder("count", None, "token", train_dataset1, 1)
model_builder1c = mc_u.create_model_builder1_LR(NUM_CLASSES, encoder_count_tokens1,tokens_input_len)

In [69]:
tuner = kt.Hyperband(model_builder1c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model1c_lr',
                     project_name='model1c_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [70]:
tuner.search(train_dataset1,
             epochs=30,
             validation_data=test_dataset1,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 00m 26s]
val_accuracy: 0.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 08m 11s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [71]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
32
0.1
0.3


Train with best hyperparameters

In [72]:
NUM_EPOCHS = 80
model1c = mc_u.create_model1_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_count_tokens, tokens_input_len)
model1c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [73]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_only_tokens",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model1c.fit(train_dataset1, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset1,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
335/346 [============================>.] - ETA: 0s - loss: 0.6727 - accuracy: 0.1310 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


346/346 [==============================] - 3s 6ms/step - loss: 0.6719 - accuracy: 0.1272 - recall_1: 0.0000e+00 - val_loss: 0.6503 - val_accuracy: 0.9359 - val_recall_1: 0.0000e+00
Epoch 2/80
338/346 [============================>.] - ETA: 0s - loss: 0.6112 - accuracy: 0.2841 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


346/346 [==============================] - 2s 6ms/step - loss: 0.6103 - accuracy: 0.2778 - recall_1: 0.0000e+00 - val_loss: 0.5744 - val_accuracy: 1.0000 - val_recall_1: 0.0000e+00
Epoch 3/80
338/346 [============================>.] - ETA: 0s - loss: 0.5198 - accuracy: 0.3466 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


...

346/346 [==============================] - 2s 6ms/step - loss: 0.1491 - accuracy: 0.4346 - recall_1: 0.0050 - val_loss: 0.1346 - val_accuracy: 0.8030 - val_recall_1: 0.0000e+00
Epoch 78/80
332/346 [===========================>..] - ETA: 0s - loss: 0.1483 - accuracy: 0.4504 - recall_1: 0.0057INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


346/346 [==============================] - 2s 6ms/step - loss: 0.1492 - accuracy: 0.4382 - recall_1: 0.0055 - val_loss: 0.1345 - val_accuracy: 0.8030 - val_recall_1: 0.0000e+00
Epoch 79/80
339/346 [============================>.] - ETA: 0s - loss: 0.1488 - accuracy: 0.4423 - recall_1: 0.0046INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_010835-9xjfqfmx/files/model-best/assets


346/346 [==============================] - 2s 6ms/step - loss: 0.1493 - accuracy: 0.4363 - recall_1: 0.0045 - val_loss: 0.1345 - val_accuracy: 0.8030 - val_recall_1: 0.0000e+00
Epoch 80/80
346/346 [==============================] - 1s 4ms/step - loss: 0.1493 - accuracy: 0.4330 - recall_1: 0.0049 - val_loss: 0.1345 - val_accuracy: 0.8030 - val_recall_1: 0.0000e+00


##### Model 2: ONLY TYPES
Find best hyperparameters

In [74]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_types = create_encoder("count", None, "type", train_dataset2, 2)
model_builder2c = mc_u.create_model_builder2_LR(NUM_CLASSES, encoder_count_types, type_input_len)

In [75]:
tuner = kt.Hyperband(model_builder2c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model2c_lr',
                     project_name='model2c_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [76]:
tuner.search(train_dataset2,
             epochs=30,
             validation_data=test_dataset2,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 00m 23s]
val_accuracy: 0.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 04m 25s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [77]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
32
0.1
0.3


Train with best hyperparameters

In [78]:
NUM_EPOCHS = 80
model2c = mc_u.create_model2_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_count_types, type_input_len)
model2c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [79]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_only_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model2c.fit(train_dataset2, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset2,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
343/346 [============================>.] - ETA: 0s - loss: 0.6688 - accuracy: 0.0800 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


346/346 [==============================] - 2s 5ms/step - loss: 0.6685 - accuracy: 0.0804 - recall_1: 0.0000e+00 - val_loss: 0.6434 - val_accuracy: 0.0025 - val_recall_1: 0.0000e+00
Epoch 2/80
335/346 [============================>.] - ETA: 0s - loss: 0.5967 - accuracy: 0.2382 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


346/346 [==============================] - 2s 5ms/step - loss: 0.5953 - accuracy: 0.2373 - recall_1: 0.0000e+00 - val_loss: 0.5551 - val_accuracy: 0.0021 - val_recall_1: 0.0000e+00
Epoch 3/80
331/346 [===========================>..] - ETA: 0s - loss: 0.4923 - accuracy: 0.3021 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


...

346/346 [==============================] - 2s 7ms/step - loss: 0.1516 - accuracy: 0.3145 - recall_1: 0.0000e+00 - val_loss: 0.1402 - val_accuracy: 0.5580 - val_recall_1: 0.0000e+00
Epoch 18/80
330/346 [===========================>..] - ETA: 0s - loss: 0.1505 - accuracy: 0.3223 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


346/346 [==============================] - 2s 5ms/step - loss: 0.1517 - accuracy: 0.3148 - recall_1: 0.0000e+00 - val_loss: 0.1401 - val_accuracy: 0.5580 - val_recall_1: 0.0000e+00
Epoch 19/80
343/346 [============================>.] - ETA: 0s - loss: 0.1511 - accuracy: 0.3125 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_011625-cal46zsv/files/model-best/assets


346/346 [==============================] - 2s 5ms/step - loss: 0.1513 - accuracy: 0.3112 - recall_1: 0.0000e+00 - val_loss: 0.1401 - val_accuracy: 0.5580 - val_recall_1: 0.0000e+00
Epoch 20/80
346/346 [==============================] - 1s 2ms/step - loss: 0.1512 - accuracy: 0.3169 - recall_1: 0.0000e+00 - val_loss: 0.1401 - val_accuracy: 0.5530 - val_recall_1: 0.0000e+00
Epoch 21/80
346/346 [==============================] - 1s 2ms/step - loss: 0.1512 - accuracy: 0.3194 - recall_1: 0.0000e+00 - val_loss: 0.1401 - val_accuracy: 0.5530 - val_recall_1: 0.0000e+00
Epoch 22/80
346/346 [==============================] - 1s 2ms/step - loss: 0.1514 - accuracy: 0.3102 - recall_1: 0.0000e+00 - val_loss: 0.1402 - val_accuracy: 0.5526 - val_recall_1: 0.0000e+00
Epoch 23/80
346/346 [==============================] - 1s 2ms/step - loss: 0.1512 - accuracy: 0.3138 - recall_1: 0.0000e+00 - val_loss: 0.1402 - val_accuracy: 0.5526 - val_recall_1: 0.0000e+00
Epoch 24/80
346/346 [==========================

##### Model 3: TOKENS AND TYPES
Find best hyperparameters

In [80]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens3 = create_encoder("count", None, "token", train_dataset3, 3)
encoder_count_types3 = create_encoder("count", None, "type", train_dataset3, 3)
model_builder3c = mc_u.create_model_builder3_LR(NUM_CLASSES, encoder_count_tokens3,encoder_count_types3, tokens_input_len, type_input_len)

In [81]:
tuner = kt.Hyperband(model_builder3c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model3c_lr',
                     project_name='model3c_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [82]:
tuner.search(train_dataset3,
             epochs=30,
             validation_data=test_dataset3,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 00m 26s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.9054276347160339
Total elapsed time: 00h 09m 31s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [83]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
32
0.1
0.2


Train with best hyperparameters

In [84]:
NUM_EPOCHS = 80
model3c = mc_u.create_model3_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_count_tokens3, encoder_count_types3, tokens_input_len, type_input_len)
model3c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [85]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_tokens_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model3c.fit(train_dataset3, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset3,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
334/346 [===========================>..] - ETA: 0s - loss: 0.6675 - accuracy: 0.0111 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


346/346 [==============================] - 3s 8ms/step - loss: 0.6665 - accuracy: 0.0109 - recall_1: 0.0000e+00 - val_loss: 0.6448 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/80
342/346 [============================>.] - ETA: 0s - loss: 0.5967 - accuracy: 0.0557 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


346/346 [==============================] - 3s 7ms/step - loss: 0.5962 - accuracy: 0.0555 - recall_1: 0.0000e+00 - val_loss: 0.5641 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/80
334/346 [===========================>..] - ETA: 0s - loss: 0.4979 - accuracy: 0.1564 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


...

346/346 [==============================] - 3s 8ms/step - loss: 0.1489 - accuracy: 0.3855 - recall_1: 9.8812e-04 - val_loss: 0.1382 - val_accuracy: 0.9050 - val_recall_1: 0.0000e+00
Epoch 79/80
344/346 [============================>.] - ETA: 0s - loss: 0.1486 - accuracy: 0.3869 - recall_1: 0.0011INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


346/346 [==============================] - 3s 8ms/step - loss: 0.1488 - accuracy: 0.3849 - recall_1: 0.0011 - val_loss: 0.1381 - val_accuracy: 0.9050 - val_recall_1: 0.0000e+00
Epoch 80/80
336/346 [============================>.] - ETA: 0s - loss: 0.1481 - accuracy: 0.3927 - recall_1: 0.0015INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_012731-6scdl5vw/files/model-best/assets


346/346 [==============================] - 3s 8ms/step - loss: 0.1487 - accuracy: 0.3831 - recall_1: 0.0014 - val_loss: 0.1381 - val_accuracy: 0.9050 - val_recall_1: 0.0000e+00


##### Model 4: TOKENS, TYPES, SEM MATH LABELS
Find best hyperparameters

In [86]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens4 = create_encoder("count", None, "token", train_dataset4, 4)
encoder_count_types4 = create_encoder("count", None, "type", train_dataset4, 4)
model_builder4c = mc_u.create_model_builder4_LR(NUM_CLASSES, encoder_count_tokens4, encoder_count_types4, tokens_input_len, type_input_len, "float")

In [87]:
tuner = kt.Hyperband(model_builder4c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model4c_lr',
                     project_name='model4c_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [88]:
tuner.search(train_dataset4,
             epochs=30,
             validation_data=test_dataset4,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 00m 31s]
val_accuracy: 0.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 08m 58s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [89]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
32
0.2
0.3


Train with best hyperparameters

In [90]:
NUM_EPOCHS = 80
model4c = mc_u.create_model4_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_count_tokens4, encoder_count_types4, tokens_input_len, type_input_len, "float")
model4c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [91]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_all_features",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model4c.fit(train_dataset4, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset4,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
337/346 [============================>.] - ETA: 0s - loss: 0.6710 - accuracy: 0.1179 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


346/346 [==============================] - 3s 8ms/step - loss: 0.6703 - accuracy: 0.1176 - recall_1: 0.0000e+00 - val_loss: 0.6510 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/80
344/346 [============================>.] - ETA: 0s - loss: 0.6049 - accuracy: 0.0940 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


346/346 [==============================] - 3s 7ms/step - loss: 0.6046 - accuracy: 0.0945 - recall_1: 0.0000e+00 - val_loss: 0.5779 - val_accuracy: 0.9058 - val_recall_1: 0.0000e+00
Epoch 3/80
336/346 [============================>.] - ETA: 0s - loss: 0.5080 - accuracy: 0.2183 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


...

346/346 [==============================] - 3s 7ms/step - loss: 0.1502 - accuracy: 0.3737 - recall_1: 0.0038 - val_loss: 0.1427 - val_accuracy: 0.9050 - val_recall_1: 0.0000e+00
Epoch 77/80
346/346 [==============================] - 1s 4ms/step - loss: 0.1503 - accuracy: 0.3766 - recall_1: 0.0038 - val_loss: 0.1427 - val_accuracy: 0.9050 - val_recall_1: 0.0000e+00
Epoch 78/80
341/346 [============================>.] - ETA: 0s - loss: 0.1499 - accuracy: 0.3808 - recall_1: 0.0039INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


346/346 [==============================] - 3s 8ms/step - loss: 0.1502 - accuracy: 0.3762 - recall_1: 0.0039 - val_loss: 0.1426 - val_accuracy: 0.9050 - val_recall_1: 0.0000e+00
Epoch 79/80
346/346 [==============================] - 1s 4ms/step - loss: 0.1502 - accuracy: 0.3762 - recall_1: 0.0037 - val_loss: 0.1426 - val_accuracy: 0.9050 - val_recall_1: 0.0000e+00
Epoch 80/80
340/346 [============================>.] - ETA: 0s - loss: 0.1495 - accuracy: 0.3817 - recall_1: 0.0039INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230316_014017-27kqak7e/files/model-best/assets


346/346 [==============================] - 3s 8ms/step - loss: 0.1501 - accuracy: 0.3764 - recall_1: 0.0039 - val_loss: 0.1426 - val_accuracy: 0.9050 - val_recall_1: 0.0000e+00
